# Predicting ad click-through with a decision tree 

We will use the dataset from a Kaggle machine learning competition, Click-Through Rate Prediction (https://www.kaggle.com/c/avazu-ctr-prediction). The dataset can be downloaded from https://www.kaggle.com/c/avazu-ctr-prediction/data.
Only the train.gz file contains labeled samples, so we only need to download this and unzip it (it will take a while)



Values are anonymized and hashed because they are categorical features, and each of their possible values corresponds to a real and meaningful value, but it is presented this way due to the privacy policy. Possibly, C1 means user gender, and 1005 and 1002 represent male and female, respectively.



Now, let’s start by reading the dataset using pandas. That’s right, pandas is extremely good at handling data in a tabular format:



In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

In [2]:
# Load data
n_rows =300000
df = pd.read_csv("~/Downloads/train.csv",nrows=n_rows)

The first 300,000 lines of the file are loaded and stored in a DataFrame. Take a quick look at the first five rows of the DataFrame:



In [3]:
print(df.head(5))

             id  click      hour    C1  banner_pos   site_id site_domain  \
0  1.000009e+18      0  14102100  1005           0  1fbe01fe    f3845767   
1  1.000017e+19      0  14102100  1005           0  1fbe01fe    f3845767   
2  1.000037e+19      0  14102100  1005           0  1fbe01fe    f3845767   
3  1.000064e+19      0  14102100  1005           0  1fbe01fe    f3845767   
4  1.000068e+19      0  14102100  1005           1  fe8cc448    9166c161   

  site_category    app_id app_domain  ... device_type device_conn_type    C14  \
0      28905ebd  ecad2386   7801e8d9  ...           1                2  15706   
1      28905ebd  ecad2386   7801e8d9  ...           1                0  15704   
2      28905ebd  ecad2386   7801e8d9  ...           1                0  15704   
3      28905ebd  ecad2386   7801e8d9  ...           1                0  15706   
4      0569f928  ecad2386   7801e8d9  ...           1                0  18993   

   C15  C16   C17  C18  C19     C20  C21  
0  320   50  

The target variable is the click column:

In [4]:
Y = df['click'].values

For the remaining columns, there are several columns that should be removed from the features (id, hour, device_id, and device_ip) as they do not contain much useful information:



In [5]:
X = df.drop(['click','id','hour','device_id','device_ip'],axis=1).values 
print(X.shape)

(300000, 19)


Each sample has 19 predictive attributes.

Next, we need to split the data into training and testing sets. Normally, we do this by randomly picking samples. However, in our case, the samples are in chronological order, as indicated in the hour field. Obviously, we cannot use future samples to predict past ones. Hence, we take the first 90% as training samples and the rest as testing samples:



In [6]:
n_train = int(n_rows * 0.9)
X_train = X[:n_train]
Y_train = Y[:n_train]
X_test = X[n_train:]
Y_test = Y[n_train:]


We initialize a OneHotEncoder object as follows:


In [7]:
enc = OneHotEncoder(handle_unknown='ignore')

In [8]:
X_train_enc = enc.fit_transform(X_train)
X_train_enc[0]


<1x8204 sparse matrix of type '<class 'numpy.float64'>'
	with 19 stored elements in Compressed Sparse Row format>

In [9]:
print(X_train_enc[0])

  (0, 2)	1.0
  (0, 6)	1.0
  (0, 188)	1.0
  (0, 2608)	1.0
  (0, 2679)	1.0
  (0, 3771)	1.0
  (0, 3885)	1.0
  (0, 3929)	1.0
  (0, 4879)	1.0
  (0, 7315)	1.0
  (0, 7319)	1.0
  (0, 7475)	1.0
  (0, 7824)	1.0
  (0, 7828)	1.0
  (0, 7869)	1.0
  (0, 7977)	1.0
  (0, 7982)	1.0
  (0, 8021)	1.0
  (0, 8189)	1.0


Each converted sample is a sparse vector.

We transform the testing set using the trained one-hot encoder as follows:



In [10]:
X_test_enc = enc.transform(X_test)

We specified the handle_unknown='ignore' parameter in the one-hot encoder to prevent errors due to any unseen categorical values


In [11]:
parameters = {'max_depth': [3, 10, None]}

We pick three options for the maximal depth – 3, 10, and unbounded. We initialize a decision tree model with Gini Impurity as the metric and 30 as the minimum number of samples required to split further:


In [12]:
decision_tree = DecisionTreeClassifier(criterion='gini',
                                        min_samples_split=30)

The classification metric should be the AUC(Area Under Curve) of the ROC(Receiver Operator Characteristic)curve, as it is an imbalanced binary case (only 51,211 out of 300,000 training samples are clicks, which is a 17% positive CTR(Click-Through Rate). As for grid search, we use three-fold (as the training set is relatively small) cross-validation and select the best-performing hyperparameter measured by the AUC:



In [13]:
grid_search = GridSearchCV(decision_tree,parameters,n_jobs=-1,cv=3,scoring='roc_auc')

**Note:** n_jobs=-1 means that we use all of the available CPU processors:

In [14]:
grid_search.fit(X_train_enc, Y_train)
print(grid_search.best_params_)

{'max_depth': 10}


We use the model with the optimal parameter to predict any future test cases as follows:

In [15]:
decision_tree_best = grid_search.best_estimator_
pos_prob = decision_tree_best.predict_proba(X_test_enc)[:, 1]

In [16]:
print(f'The ROC AUC on testing set is: {roc_auc_score(Y_test, pos_prob):.3f}')

The ROC AUC on testing set is: 0.719


The AUC we can achieve with the optimal decision tree model is 0.72. This does not seem to be very high, but click-through involves many intricate human factors, which is why predicting it is not an easy task. Although we can further optimize the hyperparameters, an AUC of 0.72 is actually pretty good. As a comparison, randomly selecting 17% of the samples to be clicked on will generate an AUC of 0.499:


In [17]:
pos_prob = np.zeros(len(Y_test))
click_index = np.random.choice(len(Y_test),int(len(Y_test)* 51211.0/300000),replace=False)
pos_prob[click_index]=1

In [18]:
print(f'The ROC AUC on testing set using random selection is:{roc_auc_score(Y_test,pos_prob):.3f}')

The ROC AUC on testing set using random selection is:0.499


Our decision tree model significantly outperforms the random predictor. Looking back, we can see that a decision tree is a sequence of greedy searches for the best splitting point at each step, based on the training dataset. However, this tends to cause overfitting as it is likely that the optimal points only work well for the training samples. Fortunately, ensembling is the technique to correct this, and random forest is an ensemble tree model that usually outperforms a simple decision tree.



# Ensembling decision trees – random forests



Random forest is a variant of the tree bagging model with additional feature-based bagging.

To employ random forest in our click-through prediction project, we can use the package from scikit-learn. Similarly to the way we implemented the decision tree in the preceding section, we only tweak the **max_depth** parameter:

In [19]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators=100,
                                       criterion='gini',
                                       min_samples_split=30,
                                       n_jobs=-1)

Besides **max_depth**, **min_samples_split**, and **class_weight**, which are important hyperparameters related to a single decision tree, hyperparameters that are related to a random forest (a set of trees) such as **n_estimators** are also highly recommended. We fine-tune **max_depth** as follows:



In [20]:
grid_search = GridSearchCV(random_forest,parameters,n_jobs=1,cv=3, scoring='roc_auc')
grid_search.fit(X_train_enc,Y_train)
print(grid_search.best_params_)




{'max_depth': None}


We use the model with the optimal parameter **None** for **max_depth** (the nodes are expanded until another stopping criterion is met) to predict any future unseen cases:



In [31]:
random_forest_best = grid_search.best_estimator_
pos_prob = random_forest_best.predict_proba(X_test_enc)[:, 1]
print(f'The ROC AUC on testing set using random forest is:{roc_auc_score(Y_test,pos_prob):.3f}')

The ROC AUC on testing set using random forest is:0.759


It turns out that the random forest model gives a substantial lift to the performance.
Let’s summarize several critical hyperparameters to tune:

- **max_depth:** This is the deepest individual tree. It tends to overfit if it is too deep or underfit if it is too shallow.

- **min_samples_split:** This hyperparameter represents the minimum number of samples required for further splitting at a node. Too small a value tends to cause overfitting, while too large a value is likely to introduce underfitting. 10, 30, and 50 might be good options to start with.



The preceding two hyperparameters are generally related to individual decision trees. The following two parameters are more related to a random forest or collection of trees:

- **max_features:** This parameter represents the number of features to consider for each best splitting point search. Typically,for an *m*-dimensional dataset,$$\sqrt{m}$$(rounded) is a recommended value for **max_features**. This can be specified as **max_features="sqrt"** in scikit-learn. Other options include **log2**, 20%, and 50% of the original features.
- **n_estimators:** This parameter represents the number of trees considered for majority voting. Generally speaking, the more trees, the better the performance but the longer the computation time. It is usually set as 100, 200, 500, and so on.


# Ensembling decision trees – gradient-boosted trees



Boosting, which is another ensemble technique, takes an iterative approach instead of combining multiple learners in parallel. In boosted trees, individual trees are no longer trained separately. Specifically, in **Gradient-Boosted Trees (GBT)** (also called **Gradient-Boosting Machines**), individual trees are trained in succession where a tree aims to correct the errors made by the previous tree



The random forest model builds each tree independently using a different subset of the dataset, and then combines the results at the end by majority votes or averaging.

The GBT model builds one tree at a time and combines the results along the way


GBT works by iteratively improving the ensemble’s predictions through the addition of sequentially trained decision trees, with each tree focusing on the residuals of the previous ones. Here’s how it works:

**Initialization:** The process starts with an initial simple model, often a single decision tree, which serves as the starting point for the ensemble.

**Sequential training:** Subsequent decision trees are trained sequentially, with each tree attempting to correct the errors of the previous ones. Each new tree is trained on the residuals (the differences between the actual and predicted values) of the ensemble’s predictions from the previous trees.

**Additive modeling:** Each new decision tree is added to the ensemble in a way that minimizes the overall error. The trees are typically shallow, with a limited number of nodes, to avoid overfitting and improve generalization.

**Learning rate:** GBT introduces a learning rate parameter, which controls the contribution of each tree to the ensemble. A lower learning rate leads to slower learning but can enhance the overall performance and stability of the ensemble.

**Ensemble prediction:** The final prediction is made by combining the predictions of all the trees in the ensemble.”



Let’s now take a look at the following steps. You will see how we predict clicks using GBT:

**1.** We import XGBoost and initialize a GBT model:

In [32]:
import xgboost as xgb
model = xgb.XGBClassifier(learning_rate=0.1,max_depth=10,
                         n_estimators=1000)

We set the learning rate to 0.1, which determines how fast or slow we want to proceed with learning in each step (in each tree, in GBT)

**Max_depth** for individual trees is set to 10. 

Additionally, 1,000 trees will be trained in sequence in our GBT model.

 **2.** Next, we train the GBT model on the training set we prepared previously:



In [33]:
model.fit(X_test_enc,Y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

**3.** We use the trained model to make predictions on the testing set and calculate the ROC AUC accordingly:



In [34]:
pos_prob = model.predict_proba(X_test_enc)[:, 1]
print(f'The ROC AUC on testing set using GBT is:{roc_auc_score(Y_test,pos_prob):.3f}')

The ROC AUC on testing set using GBT is:0.498


We are able to achieve 0.498 AUC using the XGBoost GBT model.